In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.optimizers import Adam


In [2]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
main_directory = os.path.dirname(parent_directory)
common_records =  pd.read_csv(main_directory + '/data/common_records.csv')
common_records.head()

,ID,From,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
0,Exp_1,2020-02-01 00:00:00+00:00,2020-02-01,1,Phase_1,2.393,7.6,6.1,5.0,4.4
1,Exp_1,2020-02-01 01:00:00+00:00,2020-02-01,2,Phase_1,2.056,8.2,6.2,5.1,4.4
2,Exp_1,2020-02-01 02:00:00+00:00,2020-02-01,3,Phase_1,2.258,8.4,6.3,5.3,4.5
3,Exp_1,2020-02-01 03:00:00+00:00,2020-02-01,4,Phase_1,2.535,8.4,6.4,5.4,4.5
4,Exp_1,2020-02-01 04:00:00+00:00,2020-02-01,5,Phase_1,2.042,8.2,6.5,5.5,4.6


In [3]:
unique_ids = common_records["ID"].unique()[:80]
common_records = common_records[common_records["ID"].isin(unique_ids)]

In [4]:
common_records.set_index('From',inplace=True)
common_records.head()

,ID,Date,Hour,Participation_Phase,Demand_kWh,Temperature,Temperature24,Temperature48,Temperature72
From,,,,,,,,,
2020-02-01 00:00:00+00:00,Exp_1,2020-02-01,1,Phase_1,2.393,7.6,6.1,5.0,4.4
2020-02-01 01:00:00+00:00,Exp_1,2020-02-01,2,Phase_1,2.056,8.2,6.2,5.1,4.4
2020-02-01 02:00:00+00:00,Exp_1,2020-02-01,3,Phase_1,2.258,8.4,6.3,5.3,4.5
2020-02-01 03:00:00+00:00,Exp_1,2020-02-01,4,Phase_1,2.535,8.4,6.4,5.4,4.5
2020-02-01 04:00:00+00:00,Exp_1,2020-02-01,5,Phase_1,2.042,8.2,6.5,5.5,4.6


75/25

In [5]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= common_records[df_features].values
y=common_records[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

In [6]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [7]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.0133 - val_loss: 0.0146
Epoch 2/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.0131 - val_loss: 0.0146
Epoch 3/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.0131 - val_loss: 0.0147
Epoch 4/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.0130 - val_loss: 0.0148
Epoch 5/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.0131 - val_loss: 0.0146
Epoch 6/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 0.0131 - val_loss: 0.0146
Epoch 7/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 0.0130 - val_loss: 0.0147
Epoch 8/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 0.0132 - val_loss: 0.0147
Epoch 9/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.0132 - val_loss: 0.0146
Epoch 10/10
17190/17190 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 0.0131 - val_loss: 0.0147


In [9]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=common_records.index[:len(y_pred)])

forecast_df = common_records.join(forecast_df, how="left")

print(forecast_df)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

2865/2865 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0117
Test Loss: 0.014732243493199348
2865/2865 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
                                ID        Date  Hour Participation_Phase  \
From                                                                       
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
...                            ...         ...   ...                 ...   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_

In [10]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/common_records/cnn_lstm_common_records_method_2_75_25.csv",index=True)

80/20

In [11]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= common_records[df_features].values
y=common_records[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [12]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [13]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [14]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.0130 - val_loss: 0.0164
Epoch 2/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.0128 - val_loss: 0.0164
Epoch 3/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 0.0127 - val_loss: 0.0164
Epoch 4/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.0128 - val_loss: 0.0164
Epoch 5/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.0127 - val_loss: 0.0164
Epoch 6/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.0127 - val_loss: 0.0164
Epoch 7/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 0.0127 - val_loss: 0.0164
Epoch 8/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.0128 - val_loss: 0.0164
Epoch 9/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.0127 - val_loss: 0.0164
Epoch 10/10
18336/18336 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.0127 - val_loss: 0.0164


In [15]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=common_records.index[:len(y_pred)])

forecast_df = common_records.join(forecast_df, how="left")

print(forecast_df)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

2292/2292 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0154
Test Loss: 0.016413573175668716
2292/2292 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
                                ID        Date  Hour Participation_Phase  \
From                                                                       
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
...                            ...         ...   ...                 ...   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_

In [16]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/common_records/cnn_lstm_common_records_method_2_80_20.csv",index=True)

50/50

In [17]:
df_features = ['Temperature']
df_target = ["Demand_kWh"]

X= common_records[df_features].values
y=common_records[df_target].values

X = X.reshape((X.shape[0], 1, X.shape[1])) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)

In [18]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[1])).reshape(X_train.shape)
X_test = scaler_X.transform(X_test.reshape(-1, X_test.shape[1])).reshape(X_test.shape)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [19]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(1, 1))) 
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mse')

c:\Users\alfre\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [20]:
history = model.fit(X_train, y_train,epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - loss: 0.0110 - val_loss: 0.0204
Epoch 2/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - loss: 0.0103 - val_loss: 0.0195
Epoch 3/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 29s 3ms/step - loss: 0.0103 - val_loss: 0.0195
Epoch 4/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 0.0103 - val_loss: 0.0195
Epoch 5/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.0103 - val_loss: 0.0201
Epoch 6/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.0104 - val_loss: 0.0198
Epoch 7/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.0104 - val_loss: 0.0193
Epoch 8/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 0.0103 - val_loss: 0.0198
Epoch 9/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 0.0104 - val_loss: 0.0194
Epoch 10/10
11460/11460 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 0.0103 - val_loss: 0.0199


In [21]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

forecast = model.predict(X_test)
y_pred = scaler_y.inverse_transform(forecast)

forecast_df = pd.DataFrame(data=y_pred, columns=['forecast_data'], index=common_records.index[:len(y_pred)])

forecast_df = common_records.join(forecast_df, how="left")

print(forecast_df)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

5730/5730 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.0203
Test Loss: 0.019885776564478874
5730/5730 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step
                                ID        Date  Hour Participation_Phase  \
From                                                                       
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
2020-02-01 00:00:00+00:00    Exp_1  2020-02-01     1             Phase_1   
...                            ...         ...   ...                 ...   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_176  2021-03-26    24             Phase_2   
2021-03-26 23:00:00+00:00  Exp_

In [22]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Temperature24","Temperature48","Temperature72","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Neural_Network/predictions/common_records/cnn_lstm_common_records_method_2_50_50.csv",index=True)